In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import logging
import random
import pkg_resources
import sklearn
from sklearn.model_selection import train_test_split
from rxnfp.tokenization import *
# from rdkit import Chem
# from rdkit.Chem import rdChemReactions
torch.cuda.is_available()
import rxnfp
#from rxnfp.models import SmilesClassificationModel
#from rxn_yields.core import SmilesTokenizer, SmilesClassificationModel

logger = logging.getLogger(__name__)
torch.cuda.is_available()
# from simpletransformers.classification import ClassificationModel, ClassificationArgs
from rxnfp.tokenization import SmilesTokenizer

In [2]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from rxnfp.tokenization import SmilesTokenizer

import os
import pickle

import argparse

from tqdm.auto import tqdm
import jupyterlab_widgets
import ipywidgets
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix

In [4]:
train = pd.read_csv('../ReactionClassification_2024/data/train.csv', delimiter=',')

In [5]:
val = pd.read_csv('../ReactionClassification_2024/data/val.csv', delimiter=',')

In [6]:
test = pd.read_csv('../ReactionClassification_2024/data/test.csv', delimiter=',')

In [9]:
train1 = pd.read_csv('../ReactionClassification_2024/data/train1.csv', delimiter=',')
train2 = pd.read_csv('../ReactionClassification_2024/data/train2.csv', delimiter=',')
train3 = pd.read_csv('../ReactionClassification_2024/data/train3.csv', delimiter=',')
train4 = pd.read_csv('../ReactionClassification_2024/data/train4.csv', delimiter=',')
train5 = pd.read_csv('../ReactionClassification_2024/data/train5.csv', delimiter=',')
train6 = pd.read_csv('../ReactionClassification_2024/data/train6.csv', delimiter=',')
train7 = pd.read_csv('../ReactionClassification_2024/data/train7.csv', delimiter=',')
train8 = pd.read_csv('../ReactionClassification_2024/data/train8.csv', delimiter=',')
train9 = pd.read_csv('../ReactionClassification_2024/data/train9.csv', delimiter=',')
train10 = pd.read_csv('../ReactionClassification_2024/data/train10.csv', delimiter=',')

### Train Embedding

In [13]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return(sum_embeddings / sum_mask)

In [14]:
model_path = 'rxn-data-from-postgresql/models/reaxys_bert/checkpoint-4356432-epoch-36'
#model_path = 'rxn-data-from-postgresql/models/reaxys_bert/checkpoint-4356432-epoch-36'

model = AutoModel.from_pretrained(model_path, output_hidden_states=True)
#tokenizer = SmilesTokenizer(vocab_file='rxn-data-from-postgresql/models/reaxys_bert/checkpoint-4235420-epoch-35/vocab.txt')
tokenizer = SmilesTokenizer(vocab_file='rxn-data-from-postgresql/models/reaxys_bert/checkpoint-4356432-epoch-36/vocab.txt')

Some weights of BertModel were not initialized from the model checkpoint at rxn-data-from-postgresql/models/reaxys_bert/checkpoint-4356432-epoch-36 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
val_sentence_embeddings = []
test_sentence_embeddings = []
batch= 64

In [13]:
for idx in range(1, 11):
    print(idx)
    train_df = globals()[f'train{idx}']
    train_reactions = train_df['reaction'].astype(str).tolist()
    sentence_embeddings = []
    batch_size = 64  # Define a suitable batch size

    for i in tqdm(range(0, len(train_reactions), batch_size), desc=f'Processing train{idx}'):
        # Tokenize sentences
        encoded_input = tokenizer(train_reactions[i:i+batch_size], padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)

        # Compute token embeddings
        with torch.no_grad():
            model_output = model(**encoded_input)

        # Perform pooling
        sentence_embeddings.extend([x.cpu().detach().numpy() for x in mean_pooling(model_output, encoded_input['attention_mask'])])

    # Assign embeddings to the DataFrame and save to files
    train_df['PreBertFP'] = sentence_embeddings
    X_train = np.array(train_df['PreBertFP'])
    np.save(f'../ReactionClassification_2024/data/PreBertFP/X_train{idx}_PreBertFP.npy', X_train)
    #train_df.to_csv(f'../ReactionClassification_2024/data/PreBertFP/train{idx}_PreBertFP.csv', index=False)

1


Processing train1:   0%|          | 0/3759 [00:00<?, ?it/s]

2


Processing train2:   0%|          | 0/3759 [00:00<?, ?it/s]

3


...

Processing train8:   0%|          | 0/3759 [00:00<?, ?it/s]

9


Processing train9:   0%|          | 0/3759 [00:00<?, ?it/s]

10


Processing train10:   0%|          | 0/3759 [00:00<?, ?it/s]

In [ ]:
#train.to_csv('data/PreBertFP/train_PreBertFP.csv', index=False)

### Test and Val Embedding

In [19]:
val_reactions = val['reaction'].astype(str).tolist()
for i in tqdm(range(0, len(val_reactions), batch)):
    #Tokenize sentences
    encoded_input = tokenizer(val_reactions[i:i+batch], padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)

    #Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    #Perform pooling. In this case, mean pooling
    val_sentence_embeddings.extend([x.cpu().detach().numpy() for x in mean_pooling(model_output, encoded_input['attention_mask'])])

  0%|          | 0/5175 [00:00<?, ?it/s]

In [20]:
val['PreBertFP'] = val_sentence_embeddings

In [21]:
X_val = np.array(val['PreBertFP'])

In [23]:
np.save('../ReactionClassification_2024/data/PreBertFP/X_val_PreBertFP.npy', X_val)

In [24]:
#val.to_csv('data/PreBertFP/val_PreBertFP.csv', index=False)

In [25]:
test_reactions = test['reaction'].astype(str).tolist()
for i in tqdm(range(0, len(test_reactions), batch)):
    #Tokenize sentences
    encoded_input = tokenizer(test_reactions[i:i+batch], padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)

    #Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    #Perform pooling. In this case, mean pooling
    test_sentence_embeddings.extend([x.cpu().detach().numpy() for x in mean_pooling(model_output, encoded_input['attention_mask'])])

  0%|          | 0/5169 [00:00<?, ?it/s]

In [26]:
test['PreBertFP'] = test_sentence_embeddings
X_test = np.array(test['PreBertFP'])

In [27]:
np.save('../ReactionClassification_2024/data/PreBertFP/X_test_PreBertFP.npy', X_test)

In [28]:
#test.to_csv('data/PreBertFP/test_PreBertFP.csv', index=False)

In [29]:
print('DONE')

DONE


### Data Load

In [28]:
X_train1 = np.load('../ReactionClassification_2024/data/PreBertFP/X_train1_PreBertFP.npy',allow_pickle=True)
X_train2 = np.load('../ReactionClassification_2024/data/PreBertFP/X_train2_PreBertFP.npy',allow_pickle=True)
X_train3 = np.load('../ReactionClassification_2024/data/PreBertFP/X_train3_PreBertFP.npy',allow_pickle=True)
X_train4 = np.load('../ReactionClassification_2024/data/PreBertFP/X_train4_PreBertFP.npy',allow_pickle=True)
X_train5 = np.load('../ReactionClassification_2024/data/PreBertFP/X_train5_PreBertFP.npy',allow_pickle=True)
X_train6 = np.load('../ReactionClassification_2024/data/PreBertFP/X_train6_PreBertFP.npy',allow_pickle=True)
X_train7 = np.load('../ReactionClassification_2024/data/PreBertFP/X_train7_PreBertFP.npy',allow_pickle=True)
X_train8 = np.load('../ReactionClassification_2024/data/PreBertFP/X_train8_PreBertFP.npy',allow_pickle=True)
X_train9 = np.load('../ReactionClassification_2024/data/PreBertFP/X_train9_PreBertFP.npy',allow_pickle=True)
X_train10 = np.load('../ReactionClassification_2024/data/PreBertFP/X_train10_PreBertFP.npy',allow_pickle=True)

In [29]:
X_train = np.concatenate([X_train1, X_train2, X_train3, X_train4, X_train5,
                          X_train6, X_train7, X_train8, X_train9, X_train10], axis=0)

In [30]:
np.save('../ReactionClassification_2024/data/PreBertFP/X_train_PreBertFP.npy', X_train)

In [5]:
X_train1 = np.vstack(X_train)
X_train = np.squeeze(X_train1)

In [6]:
#X_train = np.load('data/RXNFP/X_train_RXNFP.npy')
X_test = np.load('../ReactionClassification_2024/data/PreBertFP/X_test_PreBertFP.npy',allow_pickle=True)
X_val = np.load('../ReactionClassification_2024/data/PreBertFP/X_val_PreBertFP.npy',allow_pickle=True)

In [7]:
X_test1 = np.vstack(X_test)
X_test = np.squeeze(X_test1)

In [8]:
X_val1 = np.vstack(X_val)
X_val = np.squeeze(X_val1)

In [9]:
train = pd.read_csv('../ReactionClassification_2024/data/train.csv', delimiter=',')
test = pd.read_csv('../ReactionClassification_2024/data/test.csv', delimiter=',')
val = pd.read_csv('../ReactionClassification_2024/data/val.csv', delimiter=',')

In [10]:
y_train = train['CLASS-ID'].values
y_test = test['CLASS-ID'].values
y_val = val['CLASS-ID'].values

### Test on Val set

In [11]:
import faiss
index = faiss.IndexFlatL2(X_train.shape[1])
index.add(X_train)

In [12]:
from tqdm import tqdm
k = 1 

batch_size = 1000

num_batches = (X_val.shape[0] + batch_size - 1) // batch_size

In [13]:
D_val_1nn = np.zeros((X_val.shape[0], k), dtype=np.float32)
I_val_1nn = np.zeros((X_val.shape[0], k), dtype=np.int64)

for b in tqdm(range(num_batches), desc='FAISS Search on Val Data'):
    start = b * batch_size
    end = min((b + 1) * batch_size, X_val.shape[0])
    D, I = index.search(X_val[start:end], k)
    D_val_1nn[start:end, :] = D
    I_val_1nn[start:end, :] = I

FAISS Search on Val Data: 100%|██████████| 332/332 [2:02:11<00:00, 22.08s/it]


In [21]:
y_pred_val_1nn = np.array(y_train[I_val_1nn])

In [22]:
np.save('../ReactionClassification_2024/data/PreBertFP/y_pred_val_1nn', y_pred_val_1nn)

In [23]:
y_pred_val_1nn = np.load('../ReactionClassification_2024/data/PreBertFP/y_pred_val_1nn.npy')

In [24]:
print(f'Overall Accuracy: {accuracy_score(y_val, y_pred_val_1nn)}')

Overall Accuracy: 0.8333056546118286


### Test on Test set

In [12]:
k = 1  
batch_size = 1000  

num_batches = (X_test.shape[0] + batch_size - 1) // batch_size

In [13]:
D_test_1nn = np.zeros((X_test.shape[0], k), dtype=np.float32)
I_test_1nn = np.zeros((X_test.shape[0], k), dtype=np.int64)

for b in tqdm(range(num_batches), desc='FAISS Search on Test Data'):
    start = b * batch_size
    end = min((b + 1) * batch_size, X_test.shape[0])
    D, I = index.search(X_test[start:end], k)  
    D_test_1nn[start:end, :] = D  
    I_test_1nn[start:end, :] = I  

FAISS Search on Test Data:   0%|          | 0/331 [00:00<?, ?it/s]

In [15]:
y_pred_test_1nn = np.array(y_train[I_test_1nn])

In [31]:
np.save('../ReactionClassification_2024/data/PreBertFP/y_pred_test_1nn', y_pred_test_1nn)

In [17]:
print(f'Overall Accuracy: {accuracy_score(y_test, y_pred_test_1nn)}')

Overall Accuracy: 0.8328043287687796


In [18]:
report_test1nn = classification_report(y_test, y_pred_test_1nn, output_dict=True)
report_df = pd.DataFrame(report_test1nn).transpose()
df_test1nn = report_df[:-3].reset_index().rename(columns={'index': 'CLASS-ID'})
df_test1nn['test_support'] = df_test1nn['support'].astype(int)
df_test1nn['CLASS-ID'] = df_test1nn['CLASS-ID'].astype(int)

/opt/conda/envs/reaction-workbench-env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
train_class_support = train['CLASS-ID'].value_counts().sort_index()
train_support_df = train_class_support.reset_index()
train_support_df.columns = ['CLASS-ID', 'train_support']
train_support_df['CLASS-ID'] = train_support_df['CLASS-ID'].astype(int)

In [20]:
# Merge with df_val1nn on 'CLASS-ID'
df_test1nn_report = pd.merge(df_test1nn, train_support_df, on='CLASS-ID', how='left')
# Sort based on the number of train_support
df_test1nn_report = df_test1nn_report.sort_values(by='train_support', ascending=False)
df_test1nn_report = df_test1nn_report.drop(columns=['support'])

In [21]:
df_class = pd.read_csv('../ReactionClassification_2024/data/className.tsv', sep='\t', encoding='ISO-8859-1')

In [22]:
# Convert multi Class-ID into one
def clean_class_id(row):
    # Split the string by comma and convert to a list
    class_ids = str(row['CLASS-ID']).split(',')
    # Return the first element from the list, ensuring it's an integer
    return int(class_ids[0].strip())

# Apply the function to the 'CLASS-ID' column
df_class['CLASS-ID'] = df_class.apply(clean_class_id, axis=1)
df_class['CLASS-ID'] = df_class['CLASS-ID'].astype(int)

In [23]:
df_test1nn = pd.merge(df_test1nn_report, df_class[['CLASS-ID', 'TRANSFORM_NAME', 'TRANSFORM_ID']], on='CLASS-ID', how='left')

In [24]:
df_test1nn

,CLASS-ID,precision,recall,f1-score,test_support,train_support,TRANSFORM_NAME,TRANSFORM_ID
0,1085,0.952258,0.977890,0.964904,24966,185024,"Suzuki coupling, Suzuki-Miyaura Cross-Coupling","(ARCOUPLG)4.1.B, (AVNAMEDR)Suzuki-Miyaura"
1,432,0.892553,0.912790,0.902558,21259,164914,N-alkylation of alkylamines,(AG2ALKN)1.1.2
2,1016,0.885542,0.936480,0.910299,15507,113635,hydrolysis of carboxylic esters,(AQCLEAV1)1.1
3,60,0.906692,0.940009,0.923050,11185,85921,reduction of C-NO2 to C-NH2,(AAREDUCT)A.1
4,433,0.864972,0.874514,0.869717,7977,60554,"N-alkylation of benzenoid amines, anilines",(AG2ALKN)1.1.3
...,...,...,...,...,...,...,...,...
1294,1267,1.000000,1.000000,1.000000,1,8,Brackeen Imidazole Synthesis,(AVNAMEDR)Brackeen
1295,1800,0.000000,0.000000,0.000000,5,8,Shestakov Hydrazino Acid Synthesis,(AVNAMEDR)Shestakov
1296,500,0.000000,0.000000,0.000000,1,7,"O-propargylation of N-hydroxy amides, includin...",(AG2ALKO)2.4.1
1297,1622,0.000000,0.000000,0.000000,1,7,ListMacMillan Hydrogenation,(AVNAMEDR)List-MacMillan


In [32]:
df_test1nn.to_csv('data/PreBertFP/df_test1nn.csv', index=False)

In [25]:
# Calculate macro average for precision, recall, and f1-score
macro_precision2 = df_test1nn['precision'].mean()
macro_recall2 = df_test1nn['recall'].mean()
macro_f12 = df_test1nn['f1-score'].mean()

# Calculate weighted average for precision, recall, and f1-score
weighted_precision2 = (df_test1nn['precision'] * df_test1nn['test_support']).sum() / df_test1nn['test_support'].sum()
weighted_recall2 = (df_test1nn['recall'] * df_test1nn['test_support']).sum() / df_test1nn['test_support'].sum()
weighted_f12 = (df_test1nn['f1-score'] * df_test1nn['test_support']).sum() / df_test1nn['test_support'].sum()

In [26]:
output1 = f"""
Macro-averages:
- Precision: {macro_precision2:.4f}
- Recall: {macro_recall2:.4f}
- F1-score: {macro_f12:.4f}

Weighted-averages:
- Precision: {weighted_precision2:.4f}
- Recall: {weighted_recall2:.4f}
- F1-score: {weighted_f12:.4f}
"""

In [27]:
print(output1)


Macro-averages:
- Precision: 0.7646
- Recall: 0.7221
- F1-score: 0.7330

Weighted-averages:
- Precision: 0.8325
- Recall: 0.8328
- F1-score: 0.8319

